In [22]:
!pip install flask gevent requests pillow

In [23]:
pip install pyngrok

In [24]:
procfile ='web: gunicorn app:app'
procfiles = open("/content/Procfile","w")
procfiles.write(procfile)
procfiles.close()

In [25]:
!pip install flask-ngrok
!pip install pyngrok

In [26]:
from flask_ngrok import run_with_ngrok
from flask import Flask
from pyngrok import ngrok

In [27]:
#Set Token
ngrok.set_auth_token("23O8Qi7Ryjetyy2OmLLDxYlr1k6_5M1QyWjVCyMPUwCvQXBcU")

In [28]:
a = '''
<!doctype html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
<link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/css/bootstrap.min.css">
<script src="https://ajax.googleapis.com/ajax/libs/jquery/3.1.1/jquery.min.js" type="text/javascript"></script>
<script src="https://maxcdn.bootstrapcdn.com/bootstrap/3.3.7/js/bootstrap.min.js" type="text/javascript"></script>
  <title>Image Recognition Server</title>
  <body class style="margin:10px;padding:10px">
    <div class="page-header" id="banner">
      <div class="row">
        <div class="col-lg-8 col-md-7 col-sm-6">
          <h3>Image Recognition Server</h3>
          <p class="lead">Upload the image and find out what an animal is located on it?</p>
        </div>
      </div>
      <form action="" method=post enctype=multipart/form-data>
        <input type=file name=file>
        <input type=submit value=Upload>
    </form>
    </div>
    <p style="margin-bottom:2cm;"></p>
    <div class="row">
        <div class="col-lg-4">
          <div class="page-header">
            <h3 id="tables">Result</h3>
          </div>
          <div class="bs-component">
            <table class="table table-hover">
                <tr class="table-active">
                 
                  <th scope="col">Predict</th>
                </tr>
                <tr>
                         <td> {{label}} </td>
                  </tr>
            </table> 
        </div>
      </div>
  </body>
  '''

In [29]:
!mkdir '/content/templates'
!mkdir '/content/uploads'

mkdir: cannot create directory ‘/content/templates’: File exists
mkdir: cannot create directory ‘/content/uploads’: File exists


In [30]:
Html_file = open("/content/templates/index.html","w")
Html_file.write(a)
Html_file.close()


In [31]:
import os
import tensorflow as tf
from flask import Flask, request, redirect, url_for, send_from_directory, render_template
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array

In [32]:
from keras.models import Sequential, load_model
from werkzeug.utils import secure_filename
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from keras_preprocessing import image

In [33]:
import numpy as np

In [34]:
ALLOWED_EXTENSIONS = set(['jpg','jpeg','png'])
IMAGE_SIZE = (224,224)
UPLOAD_FOLDER = 'uploads'
model = None

API Specific Variables

In [35]:
app = Flask(__name__)
run_with_ngrok(app)
app.config['UPLOAD_FOLDER']= UPLOAD_FOLDER

In [36]:
def load_model():
  #Write Model Here
  global model
  model = ResNet50(weights="imagenet")

In [37]:
def allowed_file(filename):
  return '.' in filename and filename.split('.',1)[1] in ALLOWED_EXTENSIONS
  

In [38]:
def predict(file):
  img = image.load_img(file, target_size=IMAGE_SIZE)
  img = img_to_array(img)
  img = np.expand_dims(img, axis=0)
  img = preprocess_input(img)
  probs = model.predict(img)

  topfive_labels = " "
  for (imagenetID, label, prob) in decode_predictions(probs, top = 5)[0]:
    topfive_labels += str(label+", "+str(int(100*prob))+"%, ")
  return topfive_labels, predict


In [39]:
@app.route('/')
def template_test():
  return render_template("index.html",label="John's Chilli was delicious", imagesource='file://null')


In [40]:
@app.route('/', methods=['POST'])
def upload_file():
    
        file = request.files['file']
        
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file_path = os.path.join(app.config['UPLOAD_FOLDER'], filename)
            file.save(file_path)
            output = predict(file_path)
        return render_template("index.html", label=output)

In [41]:
if __name__=="__main__":
  load_model()
  app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
Exception in thread Thread-13:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connection.py", line 159, in _new_conn
    (self._dns_host, self.port), self.timeout, **extra_kw)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/util/connection.py", line 80, in create_connection
    raise err
  File "/usr/local/lib/python3.7/dist-packages/urllib3/util/connection.py", line 70, in create_connection
    sock.connect(sa)
ConnectionRefusedError: [Errno 111] Connection refused

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 600, in urlopen
    chunked=chunked)
  File "/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py", line 354, in _make_request
    conn.request(method, url, **httplib_request_kw)
  File "/usr/lib/python3.7/http/client.py",